In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Outline
To build a neural net from scratch, we need to go over each block and code those individually. At the end we can combine all of these to create an $L$-layer NN.

So, the steps we need to take are:
<ul>
    <li>Parameter Intialization: We need to initialize parameters $W$ and $b$</li>
    <li>Compute a forward propagation pass: This involves computing the linear pass - $Z^{[l]}=W^{[l]}A^{[l-1]}+b^{[l]}$ - and the activation $A^{[l]}=g(Z^{[l]})$ for both Sigmoid and ReLU activations</li>
    <li>Compute the loss</li>
    <li>Implement a back propagation pass</li>
    <li>Update the parameters: Here I'll code in mini Batch Gradient Descent (Which will cover both Stochastic Gradient Descent as well as Batch Gradient Descent), Momentum, RMSProp, and the king of them all, Adam</li>
</ul>

## Activation Functions
To add non-linearity to the model, activation functions are used. I'll define them now.
I'll be using ReLU (rectified linear unit) and sigmoid in an example, but I'll also define tanh and leaky ReLU.

In [48]:
def relu(Z):
    """
    Arguments:
    Z -- output of linear function Z = W*A+b
    
    Returns:
    ret -- ReLU(Z)
    Z -- input for use in backprop
    
    """
    return np.maximum(0,Z), Z

def sigmoid(Z):
    """
    Arguments:
    Z -- output of linear function Z = W*A+b
    
    Returns:
    ret -- sigmoid(Z)
    Z -- input for use in backprop
    
    """
    return 1./(1.+np.exp(-Z)), Z

def tanh(Z):
    """
    Arguments:
    Z -- output of linear function Z = W*A+b
    
    Returns:
    ret -- tanh(Z)
    Z -- input for use in backprop
    
    """
    return np.tanh(Z), Z

def leaky_relu(Z):
    """
    Arguments:
    Z -- output of linear function Z = W*A+b
    
    Returns:
    ret -- leaky_relu(Z)
    Z -- input for use in backprop
    
    """
    return np.maximum(0.01*Z, Z), Z
    

## Parameter Initialization
For passing parameter information between different functions, I'll use a dictionary `parameters`, which will store $W$ and $b$ values for each layer $l \{l:{0\le l \le L}\}$

Additionally, I'll implement random, Xavier initialization, and He initialization.

<ul>
    <li>Random Initialization: Samples values from a normal distribution, and multiplies by a small value to keep weights close to zero - regularization</li>
    <li>Xavier Initialization: random sampling is multiplied by constant $\sqrt{\frac{1}{\text{previous layer dimension}}}$</li>
    <li>He Initialization: random sampling is multiplied by constant $\sqrt{\frac{2}{\text{previous layer dimension}}}$</li>
</ul>

In [33]:
def initialize_parameters(layer_shape, initialization_method='he'):
    """
    Initializes parameters W and b of a network of shape layer_shape.
    
    Arguments:
    layer_shape -- list containing the dimensions of each network layer l
    
    Returns:
    parameters --  dictionary containing weight and bias parameters
    """
    #define dictionary
    params = {}
    
    #Obtain L
    L = len(layer_shape)
    
    #Check initialization_method
    if initialization_method == 'random':
        beta = 0.01
        for l in range(1,L):
            params["W"+str(l)] = np.random.randn(layer_shape[l], layer_shape[l-1])*beta
            params["b"+str(l)] = np.zeros([layer_shape[l], 1])
    
    elif initialization_method == 'xavier':
        for l in range(1,L):
            beta = np.sqrt(1./layer_shape[l-1])
            params["W"+str(l)] = np.random.randn(layer_shape[l], layer_shape[l-1])*beta
            params["b"+str(l)] = np.zeros([layer_shape[l], 1])
    
    elif initialization_method == 'he':
        for l in range(1,L):
            beta = np.sqrt(2./layer_shape[l-1])
            params["W"+str(l)] = np.random.randn(layer_shape[l], layer_shape[l-1])*beta
            params["b"+str(l)] = np.zeros([layer_shape[l], 1])
    else:
        raise NameError("%s is not a valid initalization method"%(initialization_method))

    return params

## Forward Propagation

Forward propagation refers to passing through the computation graph from left to right - forwards - and evaluating $Z^{[l]}=W^{[l]}A^{[l-1]}+b^{[l]}$ for each sucessive $l$ starting with $l=1$, in which case $A^{[0]}=X$, in other words, the activation fed into the first layer is simply the inputs.

To accomplish this, I'll create two functions. The first will evaluate the linear formula $Z^{[l]}=W^{[l]}A^{[l-1]}+b^{[l]}$, whereas the second will evaluate $A^{[l]} = g(Z^{[l]})$, which corresponds to evaluating the activation function.

Then `forward_prop` implements both to complete a forward propagation pass.

In order to compute the backprop later onwards, I'll need to store $A^{[l]}$,$W^{[l]}$, $b^{[l]}$ as well as $Z^{[l]}$ which I'll do in `linear cache` and `activation cache`

One of the arguments of `forward_prop` is `layer_activations`, which is a list of the activations for each layer of the neural network.

In [64]:
def forward_linear(W,A,b):
    """
    Linear part of forward propagation

    Arguments:
    W -- weight matrix
    A -- activations
    b -- bias matrix

    Returns:
    Z -- input to the layer's activation function
    linear_cache -- tuple with A, W, b for efficient backprop
    """
    Z = np.dot(W,A)+b
    
    linear_cache = (A,W,b)
    
    return Z, linear_cache

In [66]:
def forward_activation(Z, activation):
    """
    Arguments:
    Z -- Output of linear function Z = WA_prev+b
    activation -- String denoting activation function to use. One of [linear, sigmoid, relu, leaky_relu, tanh]
    
    Returns:
    A -- g(Z), where g() is the corresponding activation
    activation_cache -- the input Z, which will be fed into backprop
    """
    
    if activation == 'linear':
        A, activation_cache = Z, Z
    elif activation == 'sigmoid':
        A, activation_cache = sigmoid(Z), Z
    elif activation == 'relu':
        A, activation_cache = relu(Z), Z
    elif activation == 'leaky_relu':
        A, activation_cache = leaky_relu(Z), Z
    elif activation == 'tanh':
        A, activation_cache = tanh(Z), Z
    else:
        raise NameError('%s is not a valid activation function' %(activation))
    
    return A, activation_cache

In [69]:
def forward_prop(X, layer_activations, parameters):
    """
    Implements one pass of forward propagation
    
    Arguments:
    X -- input data
    layer_activations -- list of strings corresponding to the activations of each layer
    parameters -- output of initialize_parameters
    
    Returns:
    A - Output of activation function of the last layer
    caches - list of caches containing both linear and activation caches
    """
    #Define caches
    caches = []
    #A[0] is the input
    A = X
    L = len(parameters)//2 
    
    for l in range(1, L+1):
        A_prev = A
        W = parameters["W"+str(l)]
        b = parameters["b"+str(l)]
        Z, linear_cache = forward_linear(W, A_prev, b)
        A, activation_cache = forward_activation(Z, layer_activations[l])
        
        #Add both linear and activation cache to caches
        caches.append((linear_cache, activation_cache))

    return A, caches

## Cost Function

The cost function is the metric that a neural net aims to minimize. I'll implement cross-entropy cost, given by:

$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right))$$

Thus, we require a method of computing cost after one pass of forward propagation.

In [71]:
def cost(A_last, Y):
    """
    Arguments:
    A_last -- Post-activation value of the last layer of the network
    Y -- Groud truth vectors
    
    Returns:
    cost -- cross-entropy cost
    """
    #Get number of samples, m
    m = Y.shape[1]
    #Compute cross entropy cost
    cost = -1/m*np.sum(np.multiply(Y, np.log(AL))+np.multiply(1-Y, np.log(1-AL)))
    #Ensure appropriate dimensions
    cost = np.squeeze(cost)
    
    return cost
    

## Back Propagation
To update our parameters, we need to calculate the gradient of the loss with respect to $W$ and $b$

Just like with forward prop, I will implement two functions. One deals with the back pass for the linear part of the units and the other deals with the derivatives of the activation functions.

For the linear part, we take the derivatives of the parameters, obtaining:

$$ dW^{[l]} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} $$
$$ db^{[l]} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}$$
$$ dA^{[l-1]} = W^{[l] T} dZ^{[l]} $$

For the activation part, the backprop requires the gradient of the activation function. As such it depends on the activation used, and I'll define them for each one.

For sigmoid:

$$ \sigma{(z)} = \frac{1}{1+e^{-x}}$$
$$\frac{d\sigma{(z)}}{dz} = \sigma{(z)}(1-\sigma{(z)})$$

For ReLU:

$$\text{ReLU}(z) = \max{(0,z)}$$
$$\frac{d\text{ReLU}}{dz} = \left\{\begin{array}{ll}1 , z > 0\\0, z \le 0\end{array}\right.$$

Note that for ReLU, strictly speaking, there is a discontinuity at $z=0$, however since it is incredibly unlikely that the input to the function will every be exactly zero, it's fine to include it in  $z\le0$

For tanh:
$$\tanh{(z)} = \frac{e^{z}-e^{-z}}{e^{z}+e^{-z}}$$
$$\frac{d\tanh(z)}{dz} = 1-\tanh^2(z)$$

For leaky ReLU:
$$\text{leaky ReLU}(z) = \max(0.01z, z)$$
$$\frac{d(\text{leaky Relu}(z))}{dz} = \left\{\begin{array}{ll}1 , z > 0\\0.01, z \le0\end{array}\right.$$

So, I'll implement functions for each of these units to compute:
$$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]})$$

Additionally, to initialize backpropagation, we need $\frac{d\mathcal{L}}{dA^{[L]}}$, the gradient of the cost function with respect to the last activation output. For cross-entropy this is:
$$-\sum\limits_{i=1}^{m}\frac{y^{i}}{a^{[L](i)}} - \frac{1-y^{i}}{1-a^{[L](i)}}$$


In [72]:
def backward_linear(dZ, cache):
    """
    Arguments:
    dZ -- Gradient of cost w.r.t linear portion
    cache -- tuple coming from cached forward prop of layer l
    
    Returns:
    dA_prev -- gradient with respect to activation of previous layer
    dW -- gradient with respect to weights of current layer
    db -- gradient with respect to biases of current layer
    """
    
    #unpack cache
    A_prev, W, b = cache
    
    #Get number of samples
    m = A_prev.shape[1]
    
    dW = 1/m*np.dot(dZ, A_prev.T)
    db = 1/m*np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    
    return dA_prev, dW, db

In [106]:
def backward_activation(dA, Z, activation):
    """
    Arguments:
    dA -- post-activation gradient for current layer l 
    Z -- cached matrix from forward prop
    activation -- the activation to be used in the layer
    
    Returns:
    dZ -- gradient of cost function with respect to Z[l]
    """
    
    if activation == 'linear':
        dZ = dA
    
    elif activation == "relu":
        ret = np.copy(Z)
        ret[ret>0] = 1.
        ret[ret <= 0] = 0
        dZ = np.multiply(dA, ret)
        
    elif activation == "sigmoid":
        dZ = np.multiply(dA, sigmoid(Z)*(1-sigmoid(Z)))

    elif activation == "leaky_relu":
        ret = np.copy(Z)
        ret[ret>0] = 1.
        ret[ret <= 0] = 0.01
        dZ = np.multiply(dA, ret)

    elif activation == "tanh":
        dZ = np.multiply(dZ, 1 - tanh(Z)**2)
    
    else:
        raise NameError("%s is not a valid activation function" % (activation))

    return dZ
    

In [109]:
def backward_prop(AL, Y, caches, layer_activations):
    """
    Implement a backward propagation pass
    
    Arguments:
    AL -- output of the forward propagation
    Y -- ground truth
    caches -- list of caches containing linear_cache and activation_cache
    
    Returns:
    grads -- A dictionary with the gradients dA[l], dW[l], db[l]
    """
    
    #Define dict to store gradients for parameter update
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    #Ensure Y is the same as AL (which is essentially y_hat)
    Y = Y.reshape(AL.shape)
    
    #Initialize backprop, a.k.a derivative of cost with respect to AL
    dAL =  -(np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    grads["dA"+str(L)] = dAL

    for l in reversed(range(L)):
        current_cache = caches[L-1]
        dA_prev, dW, db = backward_prop(grads["dA"+str(l+1)],current_cache, layer_activations[l])
        grads["dA" + str(l)] = dA_prev
        grads["dW" + str(l + 1)] = dW
        grads["db" + str(l + 1)] = db
    
    return grads

## Update Parameters
The final step is to take the gradients computed in back propagation and use them to update the parameters $W$ and $b$.

The method of updating these parameters is important and there are several optimizers that do this in different ways.

Mini-Batch Gradient Descent:
$$ W:=W-\alpha dW $$
$$ b:=b-\alpha db $$

Momentum:

RMSProp:

Adam:
